In [1]:
import json
from glob import glob
from multiprocessing import Pool

import numpy as np
from tqdm import tqdm

In [2]:
train_annos = '/home/aj/data/DeepFashion2/train/annos/*.json'
vlidation_annos = '/home/aj/data/DeepFashion2/validation/annos/*.json'
# test_annos = '/data/DeepFasion2/test/annos/*.json'

In [3]:
def count_items(json_filepath: str) -> int:
    with open(json_filepath) as f:
        data = json.load(f)
    classes_info = [
        (
            data[item]['category_name'],
            data[item]['category_id'],
            int(len(data[item]['landmarks']) / 3),
        )
        for item
        in data.keys()
        if item.startswith('item')
    ]
    # num_items = len([item for item in data if item.startswith('item')])
    return json_filepath, len(classes_info), classes_info


def find_max_items(json_path):
    json_files = glob(json_path)
    max_items = 0
    max_item_filepath = ''
    classes_list = []
    with Pool(10) as p:
        for filepath, num_items, classes in tqdm(p.imap_unordered(count_items, json_files), total=len(json_files)):
            classes_list.append(classes)
            if num_items > max_items:
                max_items = num_items
                max_item_filepath = filepath
    return max_items, max_item_filepath, classes_list

In [4]:
max_items, max_item_filepath, classes_list = find_max_items(vlidation_annos)
print(f'{max_items = }\n{max_item_filepath = }')

100%|██████████| 32153/32153 [00:03<00:00, 8703.55it/s]

max_items = 5
max_item_filepath = '/home/aj/data/DeepFashion2/validation/annos/009750.json'


In [5]:
max_items, max_item_filepath, classes_list = find_max_items(train_annos)
print(f'{max_items = }\n{max_item_filepath = }')

100%|██████████| 191961/191961 [00:11<00:00, 16399.25it/s]

max_items = 8
max_item_filepath = '/home/aj/data/DeepFashion2/train/annos/095484.json'


In [6]:
tmp = set()
for class_list in tqdm(classes_list):
    for class_ in class_list:
        tmp.add(class_)

100%|██████████| 191961/191961 [00:00<00:00, 1564973.34it/s]


In [7]:
sorted(list(tmp), key=lambda x: x[1])

[('short sleeve top', 1, 25),
 ('long sleeve top', 2, 33),
 ('short sleeve outwear', 3, 31),
 ('long sleeve outwear', 4, 39),
 ('vest', 5, 15),
 ('sling', 6, 15),
 ('shorts', 7, 10),
 ('trousers', 8, 14),
 ('skirt', 9, 8),
 ('short sleeve dress', 10, 29),
 ('long sleeve dress', 11, 37),
 ('vest dress', 12, 19),
 ('sling dress', 13, 19)]

In [8]:
np.cumsum([i[2] for i in sorted(list(tmp), key=lambda x: x[1])]).astype(np.int64)

array([ 25,  58,  89, 128, 143, 158, 168, 182, 190, 219, 256, 275, 294])